In [1]:
from neural_net.unet import UNetDecoder, UNetEncoder
import torch

g:\mambaforge\envs\magnifier\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dummy_input = torch.randn(2, 12, 512, 512)

In [3]:
big_net = UNetEncoder(n_channels=12)
small_net = UNetEncoder(n_channels=12)
head = UNetDecoder(num_classes=2, n_channels=2048)

In [4]:
big_net_out = big_net(dummy_input)
small_net_out = small_net(dummy_input)

x1, x2, x3, x4, x5 = (torch.cat([b, s], axis=1) for b, s in zip(big_net_out, small_net_out))

In [5]:
head_out = head(x5, x1, x2, x3, x4)

In [6]:
head_out.shape

torch.Size([2, 2, 512, 512])